In [1]:
#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=4 #lattice dim
m=n+1 #space dim

R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
short = []
vshort = []
K = ceil(4*R)
K2 = (4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time



#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###

fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    A = A.transpose() #columns
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0



#exhaustive sublattice search
#each type of a matrix is the numbers on the diagonal
maxd = 54 #bound for diagonal entries
mindet = 31 #minimum determinant
maxdet = 54 #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]

allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

from time import time

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            inds = []
            l = i
            for j in range(n-1):
                inds.append(l%maxinds[j])
                l=floor(l/maxinds[j])
            mat = sublat_mat(tt,inds)
            cs = check_sublattice(mat)
            if cs==0:
                cs = prod(tt)
                if cs<bestv:
                    print("sublattice basis:")
                    print(mat)
                    bestv = cs
                    best = [mat]
                elif cs==bestv:
                    best.append(mat)
        print("best so far:",len(best))
    print("best:",len(best))
    print("min index:",bestv)
    tm = time()-tm
    print("total time mins:",tm/60)
    print("seconds per case:",tm/allc)
    return best




print(exh_search())




very short: 4
short: 20
forbidden: 18
forbidden without symmetries: 370
square L2 norm of forbidden: 130
small: 354
types: 826
all cases: 4247553
type: [1, 1, 1, 31] 31
cases of this type: 29791
best so far: 0
type: [1, 1, 1, 32] 32
cases of this type: 32768
best so far: 0
type: [1, 1, 1, 33] 33
cases of this type: 35937
best so far: 0
type: [1, 1, 1, 34] 34
cases of this type: 39304
best so far: 0
type: [1, 1, 1, 35] 35
cases of this type: 42875
best so far: 0
type: [1, 1, 1, 36] 36
cases of this type: 46656
best so far: 0
type: [1, 1, 1, 37] 37
cases of this type: 50653
best so far: 0
type: [1, 1, 1, 38] 38
cases of this type: 54872
best so far: 0
type: [1, 1, 1, 39] 39
cases of this type: 59319
best so far: 0
type: [1, 1, 1, 40] 40
cases of this type: 64000
best so far: 0
type: [1, 1, 1, 41] 41
cases of this type: 68921
best so far: 0
type: [1, 1, 1, 42] 42
cases of this type: 74088
best so far: 0
type: [1, 1, 1, 43] 43
cases of this type: 79507
best so far: 0
type: [1, 1, 1, 44] 44

best so far: 252
type: [1, 2, 1, 23] 46
cases of this type: 24334
best so far: 252
type: [1, 2, 1, 24] 48
cases of this type: 27648
best so far: 252
type: [1, 2, 1, 25] 50
cases of this type: 31250
best so far: 252
type: [1, 2, 1, 26] 52
cases of this type: 35152
best so far: 252
type: [1, 2, 1, 27] 54
cases of this type: 39366
best so far: 252
type: [1, 2, 2, 8] 32
cases of this type: 4096
best so far: 252
type: [1, 2, 2, 9] 36
cases of this type: 5832
best so far: 252
type: [1, 2, 2, 10] 40
cases of this type: 8000
best so far: 252
type: [1, 2, 2, 11] 44
cases of this type: 10648
best so far: 252
type: [1, 2, 2, 12] 48
cases of this type: 13824
best so far: 252
type: [1, 2, 2, 13] 52
cases of this type: 17576
best so far: 252
type: [1, 2, 3, 6] 36
cases of this type: 3888
best so far: 252
type: [1, 2, 3, 7] 42
cases of this type: 6174
best so far: 252
type: [1, 2, 3, 8] 48
cases of this type: 9216
best so far: 252
type: [1, 2, 3, 9] 54
cases of this type: 13122
best so far: 252
type:

best so far: 300
type: [1, 8, 1, 5] 40
cases of this type: 1000
best so far: 300
type: [1, 8, 1, 6] 48
cases of this type: 1728
best so far: 300
type: [1, 8, 2, 2] 32
cases of this type: 256
best so far: 300
type: [1, 8, 2, 3] 48
cases of this type: 864
best so far: 300
type: [1, 8, 3, 2] 48
cases of this type: 576
best so far: 300
type: [1, 8, 4, 1] 32
cases of this type: 128
best so far: 300
type: [1, 8, 5, 1] 40
cases of this type: 200
best so far: 300
type: [1, 8, 6, 1] 48
cases of this type: 288
best so far: 300
type: [1, 9, 1, 4] 36
cases of this type: 576
best so far: 300
type: [1, 9, 1, 5] 45
cases of this type: 1125
best so far: 300
type: [1, 9, 1, 6] 54
cases of this type: 1944
best so far: 300
type: [1, 9, 2, 2] 36
cases of this type: 288
best so far: 300
type: [1, 9, 2, 3] 54
cases of this type: 972
best so far: 300
type: [1, 9, 3, 2] 54
cases of this type: 648
best so far: 300
type: [1, 9, 4, 1] 36
cases of this type: 144
best so far: 300
type: [1, 9, 5, 1] 45
cases of thi

best so far: 300
type: [2, 1, 10, 2] 40
cases of this type: 800
best so far: 300
type: [2, 1, 11, 2] 44
cases of this type: 968
best so far: 300
type: [2, 1, 12, 2] 48
cases of this type: 1152
best so far: 300
type: [2, 1, 13, 2] 52
cases of this type: 1352
best so far: 300
type: [2, 1, 16, 1] 32
cases of this type: 256
best so far: 300
type: [2, 1, 17, 1] 34
cases of this type: 289
best so far: 300
type: [2, 1, 18, 1] 36
cases of this type: 324
best so far: 300
type: [2, 1, 19, 1] 38
cases of this type: 361
best so far: 300
type: [2, 1, 20, 1] 40
cases of this type: 400
best so far: 300
type: [2, 1, 21, 1] 42
cases of this type: 441
best so far: 300
type: [2, 1, 22, 1] 44
cases of this type: 484
best so far: 300
type: [2, 1, 23, 1] 46
cases of this type: 529
best so far: 300
type: [2, 1, 24, 1] 48
cases of this type: 576
best so far: 300
type: [2, 1, 25, 1] 50
cases of this type: 625
best so far: 300
type: [2, 1, 26, 1] 52
cases of this type: 676
best so far: 300
type: [2, 1, 27, 1] 5

best so far: 320
type: [3, 2, 1, 9] 54
cases of this type: 1458
best so far: 320
type: [3, 2, 2, 3] 36
cases of this type: 216
best so far: 320
type: [3, 2, 2, 4] 48
cases of this type: 512
best so far: 320
type: [3, 2, 3, 2] 36
cases of this type: 144
best so far: 320
type: [3, 2, 3, 3] 54
cases of this type: 486
best so far: 320
type: [3, 2, 4, 2] 48
cases of this type: 256
best so far: 320
type: [3, 2, 6, 1] 36
cases of this type: 72
best so far: 320
type: [3, 2, 7, 1] 42
cases of this type: 98
best so far: 320
type: [3, 2, 8, 1] 48
cases of this type: 128
best so far: 320
type: [3, 2, 9, 1] 54
cases of this type: 162
best so far: 320
type: [3, 3, 1, 4] 36
cases of this type: 192
best so far: 320
type: [3, 3, 1, 5] 45
cases of this type: 375
best so far: 320
type: [3, 3, 1, 6] 54
cases of this type: 648
best so far: 320
type: [3, 3, 2, 2] 36
cases of this type: 96
best so far: 320
type: [3, 3, 2, 3] 54
cases of this type: 324
best so far: 320
type: [3, 3, 3, 2] 54
cases of this type

best so far: 320
type: [7, 1, 6, 1] 42
cases of this type: 36
best so far: 320
type: [7, 1, 7, 1] 49
cases of this type: 49
best so far: 320
type: [7, 2, 1, 3] 42
cases of this type: 54
best so far: 320
type: [7, 2, 3, 1] 42
cases of this type: 18
best so far: 320
type: [7, 3, 1, 2] 42
cases of this type: 24
best so far: 320
type: [7, 3, 2, 1] 42
cases of this type: 12
best so far: 320
type: [7, 5, 1, 1] 35
cases of this type: 5
best so far: 320
type: [7, 6, 1, 1] 42
cases of this type: 6
best so far: 320
type: [7, 7, 1, 1] 49
cases of this type: 7
best so far: 320
type: [8, 1, 1, 4] 32
cases of this type: 64
best so far: 320
type: [8, 1, 1, 5] 40
cases of this type: 125
best so far: 320
type: [8, 1, 1, 6] 48
cases of this type: 216
best so far: 320
type: [8, 1, 2, 2] 32
cases of this type: 32
best so far: 320
type: [8, 1, 2, 3] 48
cases of this type: 108
best so far: 320
type: [8, 1, 3, 2] 48
cases of this type: 72
best so far: 320
type: [8, 1, 4, 1] 32
cases of this type: 16
best so 